In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import pinecone
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone
from langchain.retrievers import PineconeRetriever
from langchain.chains import LLMChain

# 모델 초기화
def init_model():
    return ChatOpenAI(model='gpt-4o', temperature=0.5)

# 벡터 DB 초기화
def init_vector_db(api_key, environment, index_name):
    pinecone.init(api_key=api_key, environment=environment)
    return Pinecone.from_existing_index(index_name, embedding_function=OpenAIEmbeddings())

# Retriever 초기화
def init_retriever(vector_db):
    return PineconeRetriever(vectorstore=vector_db)

# 약품 정보와 DUR 정보 검색
def retrieve_context(query, retriever):
    return retriever.get_relevant_documents(query)

# 프롬프트 생성
def generate_prompt(medicine_context, dur_context, query):
    prompt_template = """
    You are a helpful AI pharmacist. Your task is to provide simple and clear explanations about medicines for general people. Your explanation should cover the following:
    
    1. A brief explanation of the medicine, its effectiveness, dosage, precautions, side effects, and storage information.
    2. If the medicine is safe for pregnant women, including any contraindications, warning levels, and what they mean.
    
    Use the context provided below to answer the user's query.
    
    Context about the medicine:
    {medicine_context}
    
    Context about the pregnancy-related contraindications:
    {dur_context}
    
    User's query: {query}
    """
    return PromptTemplate(input_variables=["medicine_context", "dur_context", "query"], template=prompt_template)

# 답변 생성
def generate_answer(chain, medicine_context, dur_context, query):
    return chain.run(medicine_context=medicine_context, dur_context=dur_context, query=query)

# 전체 흐름 함수
def process_query(query, api_key, environment, medicine_index, dur_index):
    # 1. 모델 초기화
    model = init_model()

    # 2. 벡터 DB 초기화
    vector_db_medicine = init_vector_db(api_key, environment, medicine_index)
    vector_db_dur = init_vector_db(api_key, environment, dur_index)

    # 3. Retriever 초기화
    retriever_medicine = init_retriever(vector_db_medicine)
    retriever_dur = init_retriever(vector_db_dur)

    # 4. 약품 정보와 DUR 정보 검색
    medicine_context = retrieve_context(query, retriever_medicine)
    dur_context = retrieve_context(query, retriever_dur)

    # 5. 프롬프트 생성
    prompt = generate_prompt(medicine_context, dur_context, query)

    # 6. LLMChain 설정
    chain = LLMChain(prompt=prompt, llm=model)

    # 7. 답변 생성
    response = generate_answer(chain, medicine_context, dur_context, query)
    
    return response

# 예시 사용
api_key = "your-api-key"
environment = "your-environment"
medicine_index = "medicine_index"
dur_index = "dur_index"
query = "아스피린"

response = process_query(query, api_key, environment, medicine_index, dur_index)
print(response)
